In [98]:
import numpy as np

from strat import RSIRisingFalling

from BinanceExchangeSettings import BinanceTokenSettings

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest

np.set_printoptions(formatter={"float_kind":"{:0.2f}".format})

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
candles = dl_ex_candles(
    exchange="binance_usdm",
    symbol='1000SHIBUSDT',
    #symbol='PEPEUSDT',
    timeframe='30m',
    candles_to_dl=1344
    #since_datetime=datetime(2024,3,3),
    #until_datetime=datetime(2024,3,7),   
)

In [100]:
short_start = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.arange(50,76,5),
)

short_start.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
short_start.plot_signals(candles=candles)


INFO:info:Created RSI rsi_length= 14 


In [101]:
long_start = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(35,61,5),
)

long_start.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_start.plot_signals(candles=candles)

INFO:info:Created RSI rsi_length= 14 


In [102]:
backtest_settings = BacktestSettings()

# exchange_settings=ExchangeSettings(
#     asset_tick_step=6,
#     leverage_mode=1,
#     leverage_tick_step=2,
#     limit_fee_pct=0.0003,
#     market_fee_pct=0.0006,
#     max_asset_size=100.0,
#     max_leverage=50.0,
#     min_asset_size=0.001,
#     min_leverage=1.0,
#     mmr_pct=0.004,
#     position_mode=3,
#     price_tick_step=1,    
# )


# exchange_settings=ExchangeSettings(
#       asset_tick_step=3,
#       leverage_mode=1,
#       leverage_tick_step=2,
#       limit_fee_pct=0.0003,
#       market_fee_pct=0.0006,    
      
#       max_asset_size=100.0,
#       max_leverage=150.0,
#       min_asset_size=0.001,
#       min_leverage=1.0,
#       mmr_pct=0.004,
#       position_mode=3,
#       price_tick_step=1,    
#   )


exchange_settings=ExchangeSettings(
    asset_tick_step= 6,
    leverage_mode= 1,
    leverage_tick_step= 2,
    limit_fee_pct= 0.0003,
    market_fee_pct= 0.0006,
    max_asset_size= 2000.0,
    max_leverage= 150.0,
    min_asset_size= 0.000157,
    min_leverage= 1.0,
    mmr_pct= 0.004,
    position_mode= 3,
    price_tick_step= 0
)

#exchange_settings = BinanceTokenSettings('BTCUSDT')
#print(exchange_settings.asset_tick_step)  # Example usage

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1,0.25, 0.5]),
    sl_based_on_lookback=np.array([20,50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5,1.0]),
    trail_sl_when_pct=np.array([1,2]),
)

In [103]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_start
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 48
Total combinations of settings to test: 288
Total candles: 1,344
Total candles to test: 387,072


In [104]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(20)


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
287,5,47,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
263,5,23,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
261,5,21,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
260,5,20,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
259,5,19,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
258,5,18,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
257,5,17,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
256,5,16,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
255,5,15,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0
254,5,14,118.0,0,118,-95.5,0.0,-0.833,0.578,45.0,-955.0


In [105]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_start,
    dos_index=24,
    ind_set_index=0,
    plot_results=True,
    logger_bool=True,
)

INFO:info:Testing info log
INFO:trades:Testing trades log
INFO:info:Created RSI rsi_length= 14 
INFO:info:Indicator Settings            
Indicator Settings Index= 0            
rsi_length= 14            
rsi_is_blow= 35.0
INFO:info:
Dynamic Order settings index= 24        
max_equity_risk_pct=12.0        
max_trades=0        
risk_account_pct_size=3.0        
risk_reward=5.0        
sl_based_on_add_pct=0.1        
sl_based_on_lookback=20        
sl_bcb_type=3        
sl_to_be_cb_type=6        
sl_to_be_when_pct=0.0        
trail_sl_bcb_type=3        
trail_sl_by_pct=0.5        
trail_sl_when_pct=1.0
INFO:info:


INFO:info:ind_idx= 0 dos_idx= 24 bar_idx= 49 timestamp= 2024-02-16 08:30:00
DEBUG:info:Not in a pos so not checking SL Liq or TP
DEBUG:info:strategy evaluate
INFO:info:


INFO:info:ind_idx= 0 dos_idx= 24 bar_idx= 50 timestamp= 2024-02-16 09:00:00
DEBUG:info:Not in a pos so not checking SL Liq or TP
DEBUG:info:strategy evaluate
INFO:info:


INFO:info:ind_idx= 0 dos_idx= 24 bar_i

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 5.0,
    sl_based_on_add_pct = 0.001,
    sl_based_on_lookback = 20,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.005,
    trail_sl_when_pct = 0.01,
)
IndicatorSettingArrays(
    rsi_is_above = nan,
    rsi_is_below = 35.0,
    rsi_length = 14,
)


d:\Business\Crypto\bot\QuantFredom\env\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

